*Autenticación de hablante en base a entrada de audio

El objetivo de este notebook es la creación de una red neuronal convolucional capaz de distinguir entre parlantes en base al audio entrante y discernir si alguno de ellos es conocido en la base de datos.

Para conseguir esto, primero se preprocesará el audio, tras lo cual se extraerán las MFCC features, que serán las que se utilicen para la red neuronal.
Luego se cargará el dataset de voces conocidas usadas para entrenar la red neuronal junto con la nueva entrada de audio.
Tras eso, se entrenará la red neuronal con los datos de entrenamiento y se evaluará con los datos de test. Si la salida coincide con uno de los usuarios almacenados en la base de datos.

In [12]:
#load the information from data folder
import python_speech_features as mfcc
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm
from scipy.io import wavfile
import librosa
import librosa.display
from librosa.display import waveshow as waveplot
import soundfile

Calculates the delta of a given mfcc features array.

A delta is a coefficient that helps compute and recognize speech patterns much more easily using the MFCC features.

The formula used to calculate the delta is: dt=∑Nn=1n(ct+n−ct−n)2∑Nn=1n2, where ct is the current MFCC feature, n is the number of the MFCC feature and dt is the delta of the MFCC feature.

In [13]:
def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

First, it preprocesses a given audio, then it extracts the MFCC features and calculates the delta of the features.

Both the MFCC and the delta are returned in a single 2D array.

In [21]:
def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    

    """
    mfcc_feature = mfcc.mfcc(audio,rate, n_mfcc=40)  
    mfcc_feature = preprocessing.scale(mfcc_feature)

    delta = calculate_delta(mfcc_feature)

    combined = np.hstack((mfcc_feature,delta)) 

    plt.imshow(combined[:100].T, cmap = cm.jet)

    print(np.min(combined), np.max(combined), np.mean(combined), np.std(combined))"""

    mfc_coefficients = np.squeeze(librosa.feature.mfcc(y=audio, sr=rate, n_mfcc=20, hop_length=8000, n_fft=320))
    deltas = librosa.feature.delta(mfc_coefficients, mode='nearest')

    
    combined = np.hstack((mfc_coefficients, deltas))

    print(np.min(combined), np.max(combined), np.mean(combined), np.std(combined))
    print(mfc_coefficients.shape, deltas.shape, combined.shape)
    #plt.plot(combined[:100].T, cmap = cm.jet)

    return combined

with soundfile.SoundFile("audio.wav") as audio:
    waveform = audio.read(dtype="float32")
    sample_rate = audio.samplerate


extract_features(waveform, sample_rate)

/home/alumno/anaconda3/envs/arqespxd/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=320 is too small for input signal of length=2
  return f(*args, **kwargs)
/home/alumno/anaconda3/envs/arqespxd/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


-1010.14343 168.35724 -9.19962 118.17884
(3525632, 20) (3525632, 20) (3525632, 40)


array([[-1.01014343e+03,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.01014343e+03,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.01014343e+03,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-9.95543335e+02,  8.24042320e+00, -3.47104120e+00, ...,
         1.21417060e-01,  9.25187469e-02,  6.30166903e-02],
       [-1.00965955e+03,  5.87607384e-01,  3.41254920e-01, ...,
        -7.31834099e-02, -8.27649459e-02, -7.50745609e-02],
       [-1.01014343e+03,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)